# Import libraries 

In [0]:
#imports
import scipy
import math as mt
import numpy as np
import pandas as pd
import sklearn 
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

#tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#estimators
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.preprocessing import scale
#model metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#cross validation
from sklearn.model_selection import train_test_split
from math import sqrt
import random 

## Table Of Content:

* [1. Import Dataset](#Import)   
* [2. Feature engineering](#Feature)   
* [3. Cascading](#Cascading)   
    *  [3.1 Building](##Building)   
    *  [3.2 Latitude](##Latitude) 
    *  [3.3 Longitude](##Longitude) 
    *  [3.4 Floor](##Floor) 

# <a class="anchor" id="Import"> 1. Import dataset </a>

In [0]:
random.seed(203)

#Importing dataset
wifi = pd.read_csv("https://raw.githubusercontent.com/Layden23/wifi-python/master/Datasets/trainingData.csv", header = 0)
#Validation
validation = pd.read_csv("https://raw.githubusercontent.com/Layden23/wifi-python/master/Datasets/validationData.csv", header = 0)
validate = pd.read_csv("https://raw.githubusercontent.com/Layden23/wifi-python/master/Datasets/testData.csv", header = 0)
wifi = wifi.append(validation)
wifi["type"] = "train"
validate["type"] = "test"
twifi = wifi.append(validate)

In [0]:
wifi = wifi.iloc[:,0:520].drop_duplicates(subset=None, keep='first')

In [0]:
twifi.iloc[:, 0:520] = np.where(twifi.iloc[:, 0:520] <= 0,
         twifi.iloc[:, 0:520] + 105,
         twifi.iloc[:, 0:520] - 100)

In [0]:
twifi.iloc[:,0:520] = twifi.iloc[:,0:520].replace(np.r_[1:16],17)
twifi.iloc[:,0:520] = twifi.iloc[:,0:520] - 16 
twifi.iloc[:,0:520] = twifi.iloc[:,0:520].replace(-16,0)

In [0]:
validate = twifi[twifi["type"] == "test"]
wifi = twifi[twifi["type"] == "train"]


In [0]:
norm = Normalizer(norm="max").fit(wifi.iloc[:,0:520])
wifi.iloc[:,0:520] = norm.transform(wifi.iloc[:,0:520])
validate.iloc[:,0:520] = norm.transform(validate.iloc[:,0:520])
stand = MinMaxScaler().fit(wifi.iloc[:,0:520])
wifi.iloc[:,0:520] = stand.transform(wifi.iloc[:,0:520])
validate.iloc[:,0:520] = stand.transform(validate.iloc[:,0:520])


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [0]:
# wifi["LONGITUDE"] = - wifi["LONGITUDE"]
# validate["LONGITUDE"]= - validate["LONGITUDE"]

# <a class="anchor" id="Feature"> 2. Feature engineering </a>

In [0]:
# angle =np.arctan(wifi["LATITUDE"][0]/wifi["LONGITUDE"][0])
# angle = angle/mt.pi
# LONGITUDE = wifi["LONGITUDE"]*np.cos(angle) + wifi["LATITUDE"]*np.sin(angle)
# LATITUDE = wifi["LATITUDE"]*np.cos(angle) - wifi["LONGITUDE"]*np.sin(angle)
# plt.scatter(LONGITUDE,LATITUDE)
# wifi["LONGITUDE"] = LONGITUDE
# wifi["LATITUDE"] = LATITUDE
# plt.show()

In [0]:
# vlong = validate["LONGITUDE"]*np.cos(angle) +validate["LATITUDE"]*np.sin(angle)
# vlat = validate["LATITUDE"]*np.cos(angle) - validate["LONGITUDE"]*np.sin(angle)
# validate["LONGITUDE"] = vlong
# validate["LATITUDE"] = vlat
# plt.scatter(vlong,vlat)

In [0]:
swifi = []
for i in wifi.iloc[:,0:520]:
    swifi.append(sum(wifi[i]))
svalidate = []
for i in validate.iloc[:,0:520]:
    svalidate.append(sum(validate[i]))
sumwifi = pd.DataFrame({'col1': [i for i in wifi.iloc[:,0:520]] , 'col2': swifi})
sumvalidate = pd.DataFrame({'col1': [i for i in validate.iloc[:,0:520]] , 'col2': svalidate})

nosignal = np.where(sumwifi['col2']==0)
vnosignal = np.where(sumvalidate["col2"]==0)
badwaps = sumwifi.iloc[nosignal[0],:]
vadwaps = sumvalidate.iloc[vnosignal[0],:]  

In [0]:
wifi = wifi.drop(vadwaps["col1"], axis=1)
validate = validate.drop(vadwaps["col1"], axis = 1)
n = 520 - len(vadwaps)

In [0]:
n

267

# <a class="anchor" id="Cascading"> 3. Cascading </a>

## <a class="anchor" id="Building"> 3.1 Building </a>

In [0]:
#features
X = wifi.iloc[:,0:n]
pca = PCA(0.999)
pca.fit(wifi.iloc[:,0:n])
pcX = pd.DataFrame(pca.transform(X))
y = wifi["BUILDINGID"]
vx = validate.iloc[:,0:n] 
pcvx = pd.DataFrame(pca.transform(vx))
vy = validate["BUILDINGID"]


In [0]:
#Models
knn = KNeighborsClassifier(n_neighbors= 10, metric = "euclidean", weights="distance")
knnm = KNeighborsClassifier(n_neighbors= 10, metric = "manhattan", weights="distance")
knn.fit(pcX, y)
knnm.fit(pcX, y)
#Neuronal network
nmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(3, activation=tf.nn.softmax)
])
nmodel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
nmodel.fit(np.array(pcX), np.array(y), epochs=10)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/10
21048/21048 [==============================] - 2s 114us/sample - loss: 0.1509 - acc: 0.9784
Epoch 2/10
21048/21048 [==============================] - 2s 81us/sample - loss: 0.0077 - acc: 0.9980
Epoch 3/10
21048/21048 [==============================] - 2s 82us/sample - loss: 0.0053 - acc: 0.9979
Epoch 4/10
21048/21048 [==============================] - 2s 81us/sample - loss: 0.0045 - acc: 0.9978
Epoch 5/10
21048/21048 [==============================] - 2s 82us/sample - loss: 0.0044 - acc: 0.9977
Epoch 6/10
21048/21048 [==============================] - 2s 81us/sample - loss: 0.0040 - acc: 0.9979
Epoch 7/10
21048/21048 [==============================] - 2s 82us/sample - loss: 0.0039 - acc: 0.9981
Epoch 8/10
21048/21048 [==============================] - 2s 81us/sample - loss: 0.0040 - acc: 0.9980
Epoch 9/10
21048/21048 [==============================] - 2s 82us

In [0]:
rf = RandomForestClassifier(n_estimators = 200)
rf.fit(pcX, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
fpreds = rf.predict(pcvx)
knnpreds = knn.predict(pcvx)
knnmpreds = knnm.predict(pcvx)
npreds = nmodel.predict(pcvx)
nnpreds = []
for i in range(0,len(npreds)):
    nnpreds.append(np.argmax(npreds[i]))

In [0]:
# initialize list of lists 
performance = [['rf', accuracy_score(vy, fpreds), cohen_kappa_score(vy, fpreds)],
               ['kNN', accuracy_score(vy, knnpreds), cohen_kappa_score(vy, knnpreds)],
               ['kNNm', accuracy_score(vy, knnmpreds), cohen_kappa_score(vy, knnmpreds)],
              ['NN', accuracy_score(vy, nnpreds), cohen_kappa_score(vy, nnpreds)]] 
  
# Create the pandas DataFrame 
df_perf = pd.DataFrame(performance, columns = ['Model', 'Accuracy','Kappa'])
df_perf

,Model,Accuracy,Kappa
0,rf,0.763468,0.0
1,kNN,0.763468,0.0
2,kNNm,0.763468,0.0
3,NN,0.763468,0.0


In [0]:
confusion_matrix(vy, nnpreds, labels=None, sample_weight=None)

array([[3954, 1049,  176],
       [   0,    0,    0],
       [   0,    0,    0]])

In [0]:
confusion_matrix(vy, fpreds, labels=None, sample_weight=None)

array([[3954, 1049,  176],
       [   0,    0,    0],
       [   0,    0,    0]])

## <a class="anchor" id="Longitude"> 3.2 Longitude </a>

In [0]:
len(X1)

NameError: ignored

In [0]:
#Train
X1 = pcX.copy()
X1["BUILDINGID"] = wifi["BUILDINGID"]
dummy = pd.get_dummies(X1['BUILDINGID'])
X1 = pd.concat([X1, dummy], axis=1)
del X1['BUILDINGID']
y1 = wifi["LONGITUDE"]
#Test
vx1 = pcvx.copy()
vx1["BUILDINGID"] = fpreds
dummytest = pd.get_dummies(vx1['BUILDINGID'])
vx1 = pd.concat([vx1,dummytest], axis = 1)
del vx1['BUILDINGID']
vy1 = validate["LONGITUDE"]

ValueError: ignored

In [0]:
knn1 = KNeighborsRegressor(n_neighbors= 10, metric = "euclidean", weights="distance")
knnm1 = KNeighborsRegressor(n_neighbors= 10, metric = "manhattan", weights="distance")
knn1.fit(X1, y1)
knnm1.fit(X1, y1)


In [0]:
rf1 = RandomForestRegressor(n_estimators = 100)
rf1.fit(X1, y1)

In [0]:
rfpreds1 = rf1.predict(vx1)
knnpreds1 = knn1.predict(vx1)
knnmpreds1 = knnm1.predict(vx1)


In [0]:
# initialize list of lists 
performance1 = [['Forest', sqrt(mean_squared_error(vy1, rfpreds1)), r2_score(vy1, rfpreds1),mean_absolute_error(vy1, rfpreds1)],
               ['kNN', sqrt(mean_squared_error(vy1, knnpreds1)), r2_score(vy1, knnpreds1),mean_absolute_error(vy1, knnpreds1)],
                ['kNNm', sqrt(mean_squared_error(vy1, knnmpreds1)), r2_score(vy1, knnmpreds1),mean_absolute_error(vy1, knnmpreds1)]]
  
# Create the pandas DataFrame 
df_perf1 = pd.DataFrame(performance1, columns = ['Model', "RMSE","r2","MAE"])
df_perf1

## <a class="anchor" id="Latitude"> 3.3 Latitude </a>

In [0]:
#Train
X2 = pcX.copy()
X2["BUILDINGID"] = wifi["BUILDINGID"]
dummy = pd.get_dummies(X2['BUILDINGID'])
X2 = pd.concat([X2, dummy], axis=1)
del X2['BUILDINGID']
y2 = wifi["LATITUDE"]
#Test
vx2 = pcvx.copy()
vx2["BUILDINGID"] = fpreds
dummytest = pd.get_dummies(vx2['BUILDINGID'])
vx2 = pd.concat([vx2,dummytest], axis = 1)
del vx2['BUILDINGID']
vy2 = validate["LATITUDE"]

In [0]:
knn2 = KNeighborsRegressor(n_neighbors= 9, metric = "euclidean", weights="distance")
knn2.fit(X2, y2)
knnm2 = KNeighborsRegressor(n_neighbors= 9, metric = "manhattan", weights="distance")
knnm2.fit(X2, y2)

In [0]:
rf2 = RandomForestRegressor(n_estimators = 100)
rf2.fit(X2, y2)

In [0]:
rfpreds2 = rf2.predict(vx2)
knnpreds2 = knn2.predict(vx2)
knnmpreds2 = knnm2.predict(vx2)


In [0]:
# initialize list of lists 
performance2 = [['Forest', sqrt(mean_squared_error(vy2, rfpreds2)), r2_score(vy2, rfpreds2),mean_absolute_error(vy2, rfpreds2)],
               ['kNN', sqrt(mean_squared_error(vy2, knnpreds2)), r2_score(vy2, knnpreds2),mean_absolute_error(vy2, knnpreds2)],
                ['kNNm', sqrt(mean_squared_error(vy2, knnmpreds2)), r2_score(vy2, knnpreds2),mean_absolute_error(vy2, knnmpreds2)]]
  
# Create the pandas DataFrame 
print("LATITUDE")
df_perf2 = pd.DataFrame(performance2, columns = ['Model', 'RMSE',"r2","MAE"])
df_perf2

## <a class="anchor" id="FLOOR"> 3.4 Floor </a>

In [0]:
#Train
X3 = pcX.copy()
X3["BUILDINGID"] = wifi["BUILDINGID"]
dummy = pd.get_dummies(X3['BUILDINGID'])
X3 = pd.concat([X3, dummy], axis=1)
del X3['BUILDINGID']
y3 = wifi["FLOOR"]
#Test
vx3 = pcvx.copy()
vx3["BUILDINGID"] = fpreds
dummytest = pd.get_dummies(vx3['BUILDINGID'])
vx3 = pd.concat([vx3,dummytest], axis = 1)
vy3 = validate["FLOOR"]
del vx3['BUILDINGID']

In [0]:
knn3 = KNeighborsClassifier(n_neighbors= 9, metric = "euclidean", weights="distance")
knn3.fit(X3, y3)
knnm3 = KNeighborsClassifier(n_neighbors= 9, metric = "manhattan", weights="distance")
knnm3.fit(X3, y3)

In [0]:
rf3 = RandomForestClassifier(n_estimators = 100)
rf3.fit(X3, y3)

In [0]:
rfpreds3 = rf3.predict(vx3)
knnpreds3 = knn3.predict(vx3)
knnmpreds3 = knnm3.predict(vx3)

In [0]:
# initialize list of lists 
performance3 = [['rf', accuracy_score(vy3, rfpreds3), cohen_kappa_score(vy3, rfpreds3)],
               ['kNN', accuracy_score(vy3, knnpreds3), cohen_kappa_score(vy3, knnpreds3)],
                ['kNNm', accuracy_score(vy3, knnmpreds3), cohen_kappa_score(vy3, knnmpreds3)]] 
# Create the pandas DataFrame 
df_perf3 = pd.DataFrame(performance3, columns = ['Model', "Accuracy","Kappa"])
df_perf3

In [0]:
floor_matrix = confusion_matrix(vy3, knnpreds3, labels = None, sample_weight=None)

In [0]:
floor_matrix

## Plots

In [0]:
results = pd.DataFrame()
results["LATITUDE"]  = knnpreds2
results["LONGITUDE"] = knnpreds1
results["FLOOR"] = knnpreds3

results.to_csv("merged_all.csv", index = False)

In [0]:
print("MRE Lat:",np.mean(results["RE_Lat"]))
print("MRE Long:",np.mean(results["RE_Long"]))

In [0]:
fig = px.histogram(results, x="AE_Long")
fig.show()

In [0]:
fig = px.scatter(results, x="LATITUDE", y="RE_Lat")
fig.show()

In [0]:
fig = px.histogram(results, x="AE_Long")
fig.show()

In [0]:
fig = px.scatter(results, x="LONGITUDE", y="RE_Long")
fig.show()

In [0]:
#Building Results

for j in [fpreds,knnpreds,knnmpreds]:
    res_build = pd.DataFrame({"pred" : j, 'real': vy})
    kappa_build = []
    acc_build = []

    for i in range(len(res_build)):
        np.random.seed = i
        a = resample(res_build)
        acc_build.append(accuracy_score(a["pred"],a["real"]))
        kappa_build.append(cohen_kappa_score(a["pred"],a["real"]))

    
    fig, axs = plt.subplots(2)
    axs[0].hist(acc_build,bins = 100)
    axs[1].hist(kappa_build,bins = 100)
    plt.show
    print("Model:",j)
    print("Acc:",np.mean(acc_build),"±",1.96*st.stdev(acc_build))
    print("Kappa:",np.mean(kappa_build),"±",1.96*st.stdev(kappa_build))


In [0]:
#Longitude Results
for j in [rfpreds1,knnpreds1,knnmpreds1]:
    res_long = pd.DataFrame({"pred" : knnpreds1, 'real': vy1})
    mae_long = []
    r2_long = []
    rmse_long = []
    for i in range(len(res_long)):
        np.random.seed = i
        a = resample(res_long)
        mae_long.append(mean_absolute_error(a["pred"],a["real"]))
        r2_long.append(r2_score(a["pred"],a["real"]))
        rmse_long .append(sqrt(mean_squared_error(a["pred"],a["real"])))
    print("Model",j)    
    print("MAE:",np.mean(mae_long),"±",1.96*st.stdev(mae_long))
    print("r2:",np.mean(r2_long),"±",1.96*st.stdev(r2_long))
    print("RMSE:",np.mean(rmse_long),"±",1.96*st.stdev(rmse_long))
    fig, axs = plt.subplots(3)
    axs[0].hist(mae_long,bins = 100)
    axs[1].hist(r2_long,bins = 100)
    axs[2].hist(rmse_long,bins = 100)
    plt.show

In [0]:
#Latitude Results
for j in [rfpreds2,knnpreds2,knnmpreds2]:
    res_lat = pd.DataFrame({"pred" : j, 'real': vy2})
    mae_lat = []
    r2_lat = []
    rmse_lat = []
    for i in range(len(res_lat)):
        np.random.seed = i
        a = resample(res_lat)
        mae_lat.append(mean_absolute_error(a["pred"],a["real"]))
        r2_lat.append(r2_score(a["pred"],a["real"]))
        rmse_lat .append(sqrt(mean_squared_error(a["pred"],a["real"])))
    
    print("Model",j)  
    print("MAE:",np.mean(mae_lat),"±",1.96*st.stdev(mae_lat))
    print("r2:",np.mean(r2_lat),"±",1.96*st.stdev(r2_lat))
    print("RMSE:",np.mean(rmse_lat),"±",1.96*st.stdev(rmse_lat))

    fig, axs = plt.subplots(3)
    axs[0].hist(mae_lat,bins = 100)
    axs[1].hist(r2_lat,bins = 100)
    axs[2].hist(rmse_lat,bins = 100)
    plt.show

In [0]:
#Floor Results
for j in [rfpreds3,knnpreds3,knnmpreds3]:
    res_floor = pd.DataFrame({"pred" : j, 'real': vy3})
    kappa_floor = []
    acc_floor = []

    for i in range(len(res_floor)):
        np.random.seed = i
        a = resample(res_floor)
        acc_floor.append(accuracy_score(a["pred"],a["real"]))
        kappa_floor.append(cohen_kappa_score(a["pred"],a["real"]))

    print("Model",j)  
    print("Acc:",np.mean(acc_floor),"±",1.96*st.stdev(acc_floor))
    print("Kappa:",np.mean(kappa_floor),"±",1.96*st.stdev(kappa_floor))
    plt.hist(acc_build,bins = 100)
    plt.show
    plt.hist(kappa_build,bins = 100)
    plt.show

In [0]:
predvalidate = validate.copy()
validate["Location"] = validate.index
predvalidate["BUILDINGID"] = fpreds
predvalidate["LONGITUDE"] = knnpreds1
predvalidate["LATITUDE"] = knnpreds2
predvalidate["FLOOR"] = knnpreds3
predvalidate["Location"] = predvalidate.index


In [0]:
fig = px.scatter_3d(validate, x = 'LONGITUDE', y = 'LATITUDE', z = 'FLOOR', color = "BUILDINGID"")
fig.show()

In [0]:
wifi.BUILDINGID = wifi.BUILDINGID.astype('category')
fig = px.scatter_3d(wifi, x = 'LONGITUDE', y = 'LATITUDE', z = 'FLOOR', color = "BUILDINGID")
fig.show()